In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DIR_PATH = "/content/drive/MyDrive/CANSSI Competition/"

Mounted at /content/drive


In [ ]:
# Install TF-DF
!pip install tensorflow tensorflow_decision_forests
!pip install wurlitzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 28.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import re
import numpy as np
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import math
import os

In [ ]:
# Read Parquet file
train = pd.read_parquet(DIR_PATH+'train_datatrots_2013-2022.parquet', engine='pyarrow')
test = pd.read_parquet(DIR_PATH+'test_datatrots_2013-2022.parquet', engine='pyarrow')
print(train.shape)
print(test.shape)
train.head()

(1172292, 78)
(28120, 78)


,Barrier,CourseIndicator,Distance,FrontShoes,HandicapDistance,HindShoes,HorseAge,HorseID,JockeyID,RaceID,...,GoingAbbrev_H,GoingAbbrev_SO,GoingAbbrev_U,GoingAbbrev_VF,GoingID_1,GoingID_4,GoingID_9,GoingID_12,GoingID_24,RaceStartTime
0,0.277778,1.0,0.196429,0.0,0.4,0.0,0.307692,0.579670,0.021067,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
1,0.333333,1.0,0.196429,0.0,0.4,0.0,0.307692,0.661577,0.119715,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
2,0.388889,1.0,0.196429,0.0,0.4,0.0,0.307692,0.740050,0.107852,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
3,0.444444,1.0,0.196429,0.0,0.4,0.0,0.307692,0.334493,0.112166,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00
4,0.500000,1.0,0.196429,0.0,0.4,0.0,0.307692,0.609939,0.040768,0.371664,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2015-04-18 14:01:00


In [ ]:
# train_y = train['win probability']
# train_x = train.drop(columns=['win probability'])
# test_y = test['win probability']
# test_x = test.drop(columns=['win probability'])
# train_y

0          0.555556
1          0.177778
2          0.044444
3          0.000000
4          0.000000
             ...   
1200407    0.000000
1200408    0.000000
1200409    0.000000
1200410    1.000000
1200411    0.555556
Name: win probability, Length: 1172292, dtype: float64

In [ ]:
train = train.rename(columns={'win probability': 'win_prob', 'SexRestriction_C&G': 'SexRestriction_CG'})
test = test.rename(columns={'win probability': 'win_prob', 'SexRestriction_C&G': 'SexRestriction_CG'})
train.columns

Index(['Barrier', 'CourseIndicator', 'Distance', 'FrontShoes',
       'HandicapDistance', 'HindShoes', 'HorseAge', 'HorseID', 'JockeyID',
       'RaceID', 'RacePrizemoney', 'Saddlecloth', 'SireID', 'StartingLine',
       'TrackID', 'TrainerID', 'WeightCarried', 'WetnessScale',
       'AgeRestriction_3', 'AgeRestriction_4', 'AgeRestriction_5',
       'AgeRestriction_6', 'AgeRestriction_7', 'AgeRestriction_8',
       'AgeRestriction_9', 'AgeRestriction_10', 'Barrier_Flag', 'NW_Value',
       'Flag_CA', 'CourseIndicator_Flag', 'DamID_Frequency',
       'FoalingDate_Year', 'FoalingDate_Month', 'FoalingDate_Day_of_Week',
       'win_prob', 'HandicapType_Cwt', 'HandicapType_Hcp', 'HandicapType_SW',
       'Race_G1', 'Race_G2', 'Race_G3', 'Race_Evening', 'Race_Morning',
       'RacingSubType_T', 'StartType_V', 'SexRestriction_CG',
       'SexRestriction_F', 'SexRestriction_M', 'Surface_S', 'Surface_T',
       'FoalingCountry_AUT', 'FoalingCountry_BEL', 'FoalingCountry_CZE',
       'FoalingCou

Model Training

In [ ]:
# In order to convert the dataset into TensorFlow dataset, need to drop some columns
train = train.drop(columns=['RaceStartTime'])
test = test.drop(columns=['RaceStartTime'])

In [ ]:
label = 'win_prob'
classes = train[label].unique().tolist()
print(f"Label classes: {classes}")

train[label] = train[label].map(classes.index)

Label classes: [0.5555555555555556, 0.17777777777777778, 0.044444444444444446, 0.0, 0.022222222222222223, 1.0, 0.31111111111111117, 0.11111111111111112]


In [ ]:
label = 'win_prob'
classes = test[label].unique().tolist()
print(f"Label classes: {classes}")

test[label] = test[label].map(classes.index)

Label classes: [1.0, 0.5555555555555556, 0.31111111111111117, 0.17777777777777778, 0.11111111111111112, 0.044444444444444446, 0.022222222222222223, 0.0]


In [ ]:
# Convert the dataset into a TensorFlow dataset.
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train, label='win_prob')
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test, label='win_prob')

In [ ]:
# Train a Random Forest model.

# Specify the model.
model_1 = tfdf.keras.RandomForestModel(verbose=2)

# Train the model.
model_1.fit(train_ds)

Use 2 thread(s) for training
Use /tmp/tmptd0ush1a as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'Barrier': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'CourseIndicator': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'Distance': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'FrontShoes': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'HandicapDistance': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'HindShoes': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'HorseAge': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'HorseID': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'JockeyID': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'RaceID': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'RacePrizemoney': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>, 'Saddlecloth': <tf.Tensor 'data_11:0' shape=(None,) dtype=float64>, 'SireID': <tf.Tensor 'data_12:0' shape=(None,) dtype=float64>,

[INFO 23-11-07 15:48:02.2510 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-11-07 15:48:02.2511 UTC kernel.cc:774] Collect training examples
[INFO 23-11-07 15:48:02.2511 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-11-07 15:48:02.2555 UTC kernel.cc:393] Number of batches: 1173
[INFO 23-11-07 15:48:02.2555 UTC kernel.cc:394] Number of examples: 1172292
[INFO 23-11-07 15:48:03.1307 UTC kernel.cc:794] Training dataset:
Number of records: 1172292
Number of columns: 77

Number of columns by type:
	NUMERICAL: 76 (98.7013%)
	CATEGORICAL: 1 (1.2987%)

Columns:

NUMERICAL: 76 (98.7013%)
	0: "AgeRestriction_10" NUMERICAL mean:0.743067 

Model trained in 0:38:33.733429
Compiling model...
Model compiled.


In [ ]:
# Summary of the model structure.
model_1.summary()

Model: "random_forest_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
Total params: 1 (1.00 Byte)
Trainable params: 0 (0.00 Byte)
Non-trainable params: 1 (1.00 Byte)
_________________________________________________________________
Type: "RANDOM_FOREST"
Task: CLASSIFICATION
Label: "__LABEL"

Input Features (76):
	AgeRestriction_10
	AgeRestriction_3
	AgeRestriction_4
	AgeRestriction_5
	AgeRestriction_6
	AgeRestriction_7
	AgeRestriction_8
	AgeRestriction_9
	Barrier
	Barrier_Flag
	CourseIndicator
	CourseIndicator_Flag
	DamID_Frequency
	Distance
	Flag_CA
	FoalingCountry_AUT
	FoalingCountry_BEL
	FoalingCountry_CZE
	FoalingCountry_DEN
	FoalingCountry_FIN
	FoalingCountry_FR_
	FoalingCountry_GER
	FoalingCountry_HOL
	FoalingCountry_ITY
	FoalingCountry_NOR
	FoalingCountry_RUS
	FoalingCountry_SPA
	FoalingCountry_SWE
	FoalingCountry_SWI
	FoalingCountry_USA
	FoalingDate_Day_of_Week
	FoalingDate_Month
	Foaling

In [ ]:
# Evaluate the model.
model_1.compile(metrics=["accuracy"])
evaluation = model_1.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

29/29 [==============================] - 10s 278ms/step - loss: 0.0000e+00 - accuracy: 0.0761

loss: 0.0000
accuracy: 0.0761


In [ ]:
# Export the model to a SavedModel.
model_1.save("/tmp/my_saved_model")

ValueError: ignored

In [ ]:
tfdf.keras.get_all_models()

[tensorflow_decision_forests.keras.RandomForestModel,
 tensorflow_decision_forests.keras.GradientBoostedTreesModel,
 tensorflow_decision_forests.keras.CartModel,
 tensorflow_decision_forests.keras.DistributedGradientBoostedTreesModel]

In [ ]:
# Train a CART model.

# Specify the model.
model_2 = tfdf.keras.CartModel(verbose=2)

# Train the model.
model_2.fit(train_ds)

Use 2 thread(s) for training
Use /tmp/tmp0lwgr46d as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'Barrier': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'CourseIndicator': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'Distance': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'FrontShoes': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'HandicapDistance': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'HindShoes': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'HorseAge': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'HorseID': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'JockeyID': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'RaceID': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'RacePrizemoney': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>, 'Saddlecloth': <tf.Tensor 'data_11:0' shape=(None,) dtype=float64>, 'SireID': <tf.Tensor 'data_12:0' shape=(None,) dtype=float64>,

[INFO 23-11-07 17:23:51.0059 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-11-07 17:23:51.0060 UTC kernel.cc:774] Collect training examples
[INFO 23-11-07 17:23:51.0061 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-11-07 17:23:51.0066 UTC kernel.cc:393] Number of batches: 1173
[INFO 23-11-07 17:23:51.0066 UTC kernel.cc:394] Number of examples: 1172292
[INFO 23-11-07 17:23:51.8843 UTC kernel.cc:794] Training dataset:
Number of records: 1172292
Number of columns: 77

Number of columns by type:
	NUMERICAL: 76 (98.7013%)
	CATEGORICAL: 1 (1.2987%)

Columns:

NUMERICAL: 76 (98.7013%)
	0: "AgeRestriction_10" NUMERICAL mean:0.743067 

Model trained in 0:00:12.726398
Compiling model...
Model compiled.


In [ ]:
# Evaluate the model.
model_2.compile(metrics=["accuracy"])
evaluation = model_2.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

29/29 [==============================] - 4s 70ms/step - loss: 0.0000e+00 - accuracy: 0.0763

loss: 0.0000
accuracy: 0.0763


In [ ]:
# Export the model to a SavedModel.
model_2.save("/content/drive/MyDrive/CANSSI Competition/model_cart")

In [ ]:
# Train a GBoostTree model.

# Specify the model.
model_3 = tfdf.keras.GradientBoostedTreesModel(verbose=2)

# Train the model.
model_3.fit(train_ds)

Use 2 thread(s) for training
Use /tmp/tmpmt29on8h as temporary training directory
Reading training dataset...
Training tensor examples:
Features: {'Barrier': <tf.Tensor 'data:0' shape=(None,) dtype=float64>, 'CourseIndicator': <tf.Tensor 'data_1:0' shape=(None,) dtype=float64>, 'Distance': <tf.Tensor 'data_2:0' shape=(None,) dtype=float64>, 'FrontShoes': <tf.Tensor 'data_3:0' shape=(None,) dtype=float64>, 'HandicapDistance': <tf.Tensor 'data_4:0' shape=(None,) dtype=float64>, 'HindShoes': <tf.Tensor 'data_5:0' shape=(None,) dtype=float64>, 'HorseAge': <tf.Tensor 'data_6:0' shape=(None,) dtype=float64>, 'HorseID': <tf.Tensor 'data_7:0' shape=(None,) dtype=float64>, 'JockeyID': <tf.Tensor 'data_8:0' shape=(None,) dtype=float64>, 'RaceID': <tf.Tensor 'data_9:0' shape=(None,) dtype=float64>, 'RacePrizemoney': <tf.Tensor 'data_10:0' shape=(None,) dtype=float64>, 'Saddlecloth': <tf.Tensor 'data_11:0' shape=(None,) dtype=float64>, 'SireID': <tf.Tensor 'data_12:0' shape=(None,) dtype=float64>,

[INFO 23-11-07 17:30:03.7218 UTC kernel.cc:773] Start Yggdrasil model training
[INFO 23-11-07 17:30:03.7218 UTC kernel.cc:774] Collect training examples
[INFO 23-11-07 17:30:03.7219 UTC kernel.cc:787] Dataspec guide:
column_guides {
  column_name_pattern: "^__LABEL$"
  type: CATEGORICAL
  categorial {
    min_vocab_frequency: 0
    max_vocab_count: -1
  }
}
default_column_guide {
  categorial {
    max_vocab_count: 2000
  }
  discretized_numerical {
    maximum_num_bins: 255
  }
}
ignore_columns_without_guides: false
detect_numerical_as_discretized_numerical: false

[INFO 23-11-07 17:30:03.7223 UTC kernel.cc:393] Number of batches: 1173
[INFO 23-11-07 17:30:03.7223 UTC kernel.cc:394] Number of examples: 1172292
[INFO 23-11-07 17:30:04.6133 UTC kernel.cc:794] Training dataset:
Number of records: 1172292
Number of columns: 77

Number of columns by type:
	NUMERICAL: 76 (98.7013%)
	CATEGORICAL: 1 (1.2987%)

Columns:

NUMERICAL: 76 (98.7013%)
	0: "AgeRestriction_10" NUMERICAL mean:0.743067 

Model trained in 4:47:11.648307
Compiling model...
Model compiled.


In [ ]:
# Evaluate the model.
model_3.compile(metrics=["accuracy"])
evaluation = model_3.evaluate(test_ds, return_dict=True)
print()

for name, value in evaluation.items():
  print(f"{name}: {value:.4f}")

29/29 [==============================] - 6s 130ms/step - loss: 0.0000e+00 - accuracy: 0.0764

loss: 0.0000
accuracy: 0.0764


In [ ]:
# Export the model to a SavedModel.
model_3.save("/content/drive/MyDrive/CANSSI Competition/model_gradient_boost")